In [2]:
# import sys
# !python -m pip install pyspark==3.2.1



In [3]:
# !python -m pip install findspark

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:

scala_version = '2.12'
spark_version = '3.2.1'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}' , 'org.apache.kafka:kafka-clients:3.3.1' ]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [3]:
#create dataframe from Kafka data
topic_name='RandomNumber'
kafka_server= 'localhost:9092'
kafkaDf=spark.read.format("kafka").option("kafka.bootstrap.servers",kafka_server)\
    .option("subscribe",topic_name).option("startingOffsets","earliest").load()

In [4]:
#show data
kafkaDf.toPandas()

,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,0,2022-10-11 15:29:16.417,0
1,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1,2022-10-11 15:29:21.422,0
2,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,2,2022-10-11 15:29:26.437,0
3,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,3,2022-10-11 15:29:31.440,0
4,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,4,2022-10-11 15:29:36.449,0
...,...,...,...,...,...,...,...
1039,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1039,2022-10-18 09:00:17.923,0
1040,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1040,2022-10-18 09:00:22.951,0
1041,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1041,2022-10-18 09:00:27.961,0
1042,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1042,2022-10-18 09:00:32.981,0


In [5]:
from dis import dis
from time import sleep
from traceback import print_tb
from IPython.display import display,clear_output

In [6]:
#show streaming data using for loop
batchDF=kafkaDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))



for x in range(0,2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Sencond passed: {x*5}")
        display(batchDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ends ...")

Showing live view refreshed every 5 seconds
Sencond passed: 0


,topic,offset,rand_number
0,RandomNumber,0,0
1,RandomNumber,1,1
2,RandomNumber,2,2
3,RandomNumber,3,3
4,RandomNumber,4,4
...,...,...,...
1043,RandomNumber,1043,2
1044,RandomNumber,1044,2
1045,RandomNumber,1045,2
1046,RandomNumber,1046,3


break
Live view ends ...


Perform some data aggreation and show live results

In [13]:
batchCountDF = batchDF.groupBy('rand_number').count()
for x in range (0,2000):
    try:
        print('showing live view refreshed every 5 seconds')
        print(f"Second passed: {x*5}")
        display(batchCountDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended")

showing live view refreshed every 5 seconds
Second passed: 2115


,rand_number,count
0,7,66
1,3,266
2,8,66
3,0,7
4,5,66
5,6,66
6,9,56
7,1,450
8,4,164
9,2,349


break
Live view ended


Create Streaming dataframe from Kafka

In [8]:
streamRawDf = spark.readStream.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).load()
streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))
checkEvenDF = streamDF.withColumn('Is_Even',col('rand_number').cast('int') % 2 == 0 )

In [11]:
topic_name

'RandomNumber'

Write stream

In [9]:
from random import randint
randNum=str(randint(0,10000))
q1name = "queryNumber"+randNum
q2name = "queryCheckEven"+randNum
stream_writer1 = (streamDF.writeStream.queryName(q1name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
stream_writer2 = (checkEvenDF.writeStream.queryName(q2name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
query1 = stream_writer1.start()
query2 = stream_writer2.start()

View streaming result

In [12]:
for x in range(0,2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"second passed: {x*5}")
        result1=spark.sql(f"SELECT * from {query1.name}")
        result2=spark.sql(f"SELECT * from {query2.name}")
        display(result1.toPandas())
        display(result2.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view end")


Showing live view refreshed every 5 seconds
second passed: 480


,topic,offset,rand_number


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\Spark\spark-3.3.0-bin-hadoop3\python\lib\py4j-0.10.9.5-src.zip\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Spark\spark-3.3.0-bin-hadoop3\python\lib\py4j-0.10.9.5-src.zip\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "d:\anconda\envs\cs231\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


break
Live view end
